In [ ]:
import pandas as pd
import pyarrow
import pyarrow.parquet

from paths import PROCESSED_DATA_DIR, RAW_DATA_DIR
from  zip_reader import read_zip
from preprocessing import preprocess_full, preprocess_partial

In [ ]:
df = pyarrow.parquet.read_table(PROCESSED_DATA_DIR).to_pandas()

In [ ]:
df['ts_diff'] = df.sort_values(by='Timestamp').groupby('MMSI')['Timestamp'].diff()
